In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import os
import glob
from os import walk
from matplotlib.gridspec import GridSpec
from IPython.display import display, clear_output
from nano_lab import experiments
import nano_lab as lab
from mpl_toolkits.axes_grid1 import make_axes_locatable,ImageGrid
from matplotlib.gridspec import GridSpec
import matplotlib as mpl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
mpl.rcParams.update(mpl.rcParamsDefault)
plt.style.use('../src/plotstyle.mplstyle')
exp= lab.experiments('nano-lab','nsom','fg163r',False)
exp.dframe

,No. Dir,Name Dir
0,0,2022-10-20-nsom.h5
1,1,2022-10-20-nsom.h5
2,2,2022-10-20-nsom.h5
3,3,2022-10-20-nsom.h5


In [2]:
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.gridspec as gridspec
import matplotlib.animation as animation
import math
import warnings
warnings.filterwarnings("ignore")


class afm_nsom:
    def __init__(self,alldata,measure):
        self.alldata=alldata
        self.measure = measure
        self.afm_nsom_data = self.alldata[self.measure][0]
        self.xi=0;self.xf = self.afm_nsom_data.shape[1]
        self.yi=0;self.yf = self.afm_nsom_data.shape[0]
        self.afm          = self.afm_nsom_data[:,:,0]
        self.lockin       =  self.afm_nsom_data[:,:,1]
        self.multimeter   =  self.afm_nsom_data[:,:,2]
        try:
            self.amplitud = self.alldata[self.measure][1]
            self.fase = self.alldata[self.measure][2]
        except:
            pass




In [2]:
afm, lockin, multimeter = exp.afm_nsom_data(3)
attrs  = exp.exps_attr(2)
nx, ny = afm.shape[1], afm.shape[0]
x, y = np.meshgrid(np.arange(0, nx, 1), np.arange(0, ny, 1))
xx = np.arange(0, nx, 1)
yy = np.arange(0, ny, 1)
xt =np.arange(xx[0],xx[-1],7)
attrs

{'Inicio X': 0.0,
 'Inicio Y': 0.0,
 'Tipo': 'R',
 'Unidades': 'nm',
 'fin X ': 4000.0,
 'fin Y ': 4000.0,
 'paso': 100.0}

In [18]:
def zmap(exptype):
    z = exptype
    offs = np.min(z)*0.20
    if np.min(z)-offs > 1.e-4:
        z_offset=(np.min(z)-offs)*np.ones(z.shape)#
    else:
        z_offset=(np.min(z)+offs)*np.ones(z.shape)#
    x_offset=-np.min(xx)*np.ones(z.shape)
    y_offset=np.min(yy)*np.ones(z.shape)

    proj_z=lambda x, y, z: z#projection in the z-direction
    colorsurfz=proj_z(x,y,z)
    proj_x=lambda x, y, z: x
    colorsurfx=proj_z(x,y,z)
    proj_y=lambda x, y, z: y
    colorsurfy=proj_z(x,y,z)

    class Results(): pass
    results = Results()
    results.colorsurfx=  colorsurfx
    results.colorsurfy=  colorsurfy
    results.colorsurfz=  colorsurfz
    results.x_offset = x_offset
    results.y_offset = y_offset
    results.z_offset = z_offset
    return results



fig = make_subplots(
        rows=1, cols=3,
        specs=[[{'type': 'surface'}, {'type': 'surface'},{'type': 'surface'}]],print_grid=False, shared_xaxes=True,shared_yaxes=True,)

    # Generate data




exp2plots = [afm,lockin,multimeter]
for i,experiment in enumerate(exp2plots):
    fig.add_trace(
        go.Surface(z=experiment, colorscale='Plotly3',showscale=False, reversescale=True,),row=1, col=i+1)
    # fig.update_traces(contours_x=dict(show=True, usecolormap=True, project_x=True))
    # fig.update_traces(contours_y=dict(show=True, usecolormap=True, project_y=True))

    fig.add_trace(go.Surface(z=list(zmap(experiment).z_offset),  # type: ignore
                x=list(x),
                y=list(y),
                colorscale='Plotly3',showscale=False, reversescale=True,
                 surfacecolor=zmap(experiment).colorsurfz,  # type: ignore
               ),row=1, col=i+1)

    # fig.add_trace(go.Surface(z=list(zmap(experiment).x_offset),
    #             x=list(zmap(experiment).x_offset),
    #             y=list(zmap(experiment).x_offset),
    #             showlegend=False,
    #             showscale=False,
    #             surfacecolor=zmap(experiment).colorsurfx,
    #            ))

               
fig.update_layout(
     template="plotly_dark",
     width=1200,
    font=dict(
        family="Computer Modern Roman,serif",
        color='white',
        size=13,
    ),
    # scene=dict(
    # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
    # scene2=dict(
    # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
    # scene3=dict(
    # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",),
    # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt*100,tickvals=xt,tickprefix= "nm",)),
 )

fig.show()

In [ ]:
def zmap(exptype):
    z = exptype
    offs = np.mean(z)/2 
    z_offset=(np.min(z)-offs)*np.ones(z.shape)#
    x_offset=-np.min(xx)*np.ones(z.shape)
    y_offset=np.min(yy)*np.ones(z.shape)

    proj_z=lambda x, y, z: z#projection in the z-direction
    colorsurfz=proj_z(x,y,z)
    proj_x=lambda x, y, z: x
    colorsurfx=proj_z(x,y,z)
    proj_y=lambda x, y, z: y
    colorsurfy=proj_z(x,y,z)

    class Results(): pass
    results = Results()
    results.colorsurfy =  colorsurfy
    results.z_offset = z_offset
    return results



In [ ]:
def plotting(afm,lockin,multimeter,**kwargs):

    fig = make_subplots(
            rows=1, cols=3,
            specs=[[{'type': 'surface'}, {'type': 'surface'},{'type': 'surface'}]],print_grid=False, shared_xaxes=True,shared_yaxes=True,)


    if 'step' in kwargs:
        step = kwargs.get('step')
    else: 
        step =100
        # Generate data
    nx = afm.shape[0]
    ny = afm.shape[1]
    x = np.arange(0, nx, 1)
    y = np.arange(0, ny, 1)
    xt =np.arange(x[0],x[-1],7)

    exp2plots = [afm,lockin,multimeter]
    for i,expriment in enumerate(exp2plots):
        fig.add_trace(
            go.Surface( z=expriment, colorscale='Plotly3',showscale=False, reversescale=True,),row=1, col=i+1)
        fig.update_traces(contours_x=dict(show=True, usecolormap=True, project_x=True))
        fig.update_traces(contours_y=dict(show=True, usecolormap=True, project_y=True))


    fig.update_layout(
        template="plotly_dark",
        width=1200,
        font=dict(
            family="Computer Modern Roman,serif",
            color='white',
            size=13,
        ),
        scene=dict(
        xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",),
        yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",)),
        scene2=dict(
        xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",),
        yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",)),
        scene3=dict(
        xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",),
        yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=xt,tickvals=xt,tickprefix= "nm",)),
    )

    return fig


plotting(afm,lockin,multimeter)

In [ ]:
class plotts:
    def __init__(self,afm,lockin,multimeter,step=1):
        self.afm = afm
        self.lockin = lockin
        self.multimeter = multimeter
        self.step = step
        self.nx = self.afm.shape[0]
        self.ny = self.afm.shape[1]
        self.xm, self.ym = np.meshgrid(np.arange(0, self.nx, 1), np.arange(0, self.ny, 1))
        self.x = np.arange(0, nx, 1)
        self.y = np.arange(0, ny, 1)
        self.xt =np.arange(self.x[0],self.x[-1],3)
        self.yt =np.arange(self.y[0],self.y[-1],3)
        self.xtl = self.xt*self.step
        self.ytl = self.yt*self.step

    def plotting(self,**kwargs):
        fig = make_subplots(
                rows=1, cols=3,
                specs=[[{'type': 'surface'}, {'type': 'surface'},{'type': 'surface'}]],print_grid=False, shared_xaxes=True,shared_yaxes=True,)

        self.exp2plots = [self.afm,self.lockin,self.multimeter]
        for i,expriment in enumerate(self.exp2plots):
            fig.add_trace(
                go.Surface( z=expriment, colorscale='Plotly3',showscale=False, reversescale=True,),row=1, col=i+1)
            fig.update_traces(contours_x=dict(show=True, usecolormap=True, project_x=True))
            fig.update_traces(contours_y=dict(show=True, usecolormap=True, project_y=True))


        fig.update_layout(
            template="plotly_dark",
            #width=1200,
            font=dict(
                family="Computer Modern Roman,serif",
                color='white',
                size=13,
            ),
            # scene=dict(
            # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=self.xtl,tickvals=self.xt,tickprefix= "nm",),
            # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=self.ytl,tickvals=self.yt,tickprefix= "nm",)),
            # scene2=dict(
            # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=self.xtl,tickvals=self.xt,tickprefix= "nm",),
            # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=self.ytl,tickvals=self.yt,tickprefix= "nm",)),
            # scene3=dict(
            # xaxis=dict(title = 'x (nm)',tickmode='array',ticktext=self.xtl,tickvals=self.xt,tickprefix= "nm",),
            # yaxis=dict(title = 'y (nm)',tickmode='array',ticktext=self.ytl,tickvals=self.yt,tickprefix= "nm",)),
        )
        return fig

    def plotall(self,exp='afm',**kwargs):
        fig = make_subplots(
                rows=2, cols=2, 
                column_widths=[0.6, 0.4],
                row_heights=[0.6, 0.4],
                specs=[[{'type': 'surface','rowspan':2},{'type': 'contour'}],
                    [None, {'type':'scatter'}]],
                horizontal_spacing=0.03,
                vertical_spacing=0.05,
                shared_xaxes=True,
                )
            # Generate data
        if exp=='afm':
            fig.add_trace(go.Surface(z=self.afm.T, colorscale='Plotly3',showscale=False, reversescale=True),row=1, col=1)
            fig.add_trace(go.Heatmap(z=self.afm.T, colorscale='Plotly3',reversescale=True,connectgaps=True, zsmooth='best'),row=1, col=2)
            fig.add_trace(go.Scatter(x=self.xm[0],y=self.afm[:,5]),row=2,col=2)
        elif exp=='lockin':
            fig.add_trace(go.Surface(z=self.lockin.T, colorscale='Plotly3',showscale=False, reversescale=True),row=1, col=1)
            fig.add_trace(go.Heatmap(z=self.lockin.T, colorscale='Plotly3',reversescale=True,connectgaps=True, zsmooth='best'),row=1, col=2)
            fig.add_trace(go.Scatter(x=self.xm[0],y=self.lockin[:,5]),row=2,col=2)
        elif exp=='multimeter':
            fig.add_trace(go.Surface(z=self.multimeter.T, colorscale='Plotly3',showscale=False, reversescale=True),row=1, col=1)
            fig.add_trace(go.Heatmap(z=self.multimeter.T, colorscale='Plotly3',reversescale=True,connectgaps=True, zsmooth='best'),row=1, col=2)
            fig.add_trace(go.Scatter(x=self.xm[0] ,y=self.multimeter[:,5]),row=2,col=2)

        fig.update_layout(
            template="plotly_dark",
             autosize=True,
        margin=dict(l=5, r=5, b=25, t=20),
            # width=1000,
            # height=500,
            font=dict(
                family="Latin Modern Roman,serif",
                color='white',
                size=13,
            ),   
              xaxis2 = dict(title='x (nm)',tickvals=self.xt,ticktext=self.xtl)
            
        )
        return fig

plotts(afm,lockin,multimeter,step=150).plotall('lockin')


